In [2]:
import pickle
from datasets import load_dataset
import numpy as np


multi_lexsum = load_dataset("allenai/multi_lexsum", name="v20230518")
modified_dataset = multi_lexsum["test"].filter(lambda x: x["summary/short"] != None)

In [3]:
import evaluate

rouge_scoring = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

In [43]:
p = """Plaintiffs Harriet Delores Cleveland and Markis Antwuan Watts filed a class action lawsuit against the City of Montgomery, Alabama. They were represented by attorneys from the Southern Poverty Law Center. The plaintiffs alleged that the city violated their constitutional rights by imprisoning them for non-payment of fines without considering their ability to pay. On May 1, 2014, the District Court granted a preliminary injunction in favor of the plaintiffs. The parties reached a settlement agreement, resulting in changes to Municipal Court procedures. The case was closed on October 31, 2014. The plaintiffs sought a declaration that the city's actions violated their rights and requested attorneys' fees. The judges of the Montgomery Municipal Court were required to follow the constitutional principles outlined in Bearden v. Georgia and Turner v. Rogers. The case was filed in the U.S. District Court for the Middle District of Alabama."""

ex = modified_dataset["summary/short"][1]

print(rouge_scoring.compute(predictions=[p], references=[ex], use_stemmer = True))
print(bertscore.compute(predictions=[p], references=[ex], model_type="microsoft/deberta-large-mnli"))

{'rouge1': 0.55625, 'rouge2': 0.27044025157232704, 'rougeL': 0.28124999999999994, 'rougeLsum': 0.28124999999999994}
{'precision': [0.70556640625], 'recall': [0.6686365604400635], 'f1': [0.6866052746772766], 'hashcode': 'microsoft/deberta-large-mnli_L18_no-idf_version=0.3.12(hug_trans=4.38.0.dev0)'}


In [50]:
import os

## ROUGE
# ['led: rouge1: 46.23', 'led: rouge2: 22.27', 'led: rougeL: 30.18', 'led: rougeLsum: 30.76']
# ['primera: rouge1: 42.75', 'primera: rouge2: 19.41', 'primera: rougeL: 28.50', 'primera: rougeLsum: 28.95']

### BERTScore
# ['led: precision: 0.73', 'led: recall: 0.68', 'led: f1: 0.71']
# ['primera: precision: 0.70', 'primera: recall: 0.66', 'primera: f1: 0.68']

for file in os.listdir():
    if "predicted" in file:
        predicted = pickle.load(open(file, "rb"))
        print(file.split(".")[0])
        print([f"{metric} : {np.mean(val) * 100:0.2f}"for metric, val in rouge_scoring.compute(predictions=predicted, references=modified_dataset["summary/short"][:len(predicted)], use_stemmer = True).items()])
        print([f"{metric} : {np.mean(val[0]):0.2f}"for metric, val in bertscore.compute(predictions=predicted, references=modified_dataset["summary/short"][:len(predicted)], model_type="microsoft/deberta-large-mnli", batch_size = 2).items() if metric != "hashcode"])
        print("----------")


predicted_text_cot_summarization_random_selection
['rouge1 : 46.59', 'rouge2 : 26.72', 'rougeL : 35.34', 'rougeLsum : 35.34']
['precision : 0.62', 'recall : 0.71', 'f1 : 0.66']
----------
